In [1]:
import snowflake.connector
import pandas as pd
from snowflake.snowpark.session import Session

conn = {'user':"ELII",
    'password':"Elii123456789!",
    'account':"CMZNSCB-MU47932",
    'warehouse':"COMPUTE_WH",
    'database':"WELLS",
    'schema':"MINERALS"}

def get_fast_edit_wells():
    """Get all wells with FAST_EDIT=1"""
    # try:
    session = Session.builder.configs(conn).create()
    
    # Join ECON_INPUT with vw_well to get well names and trajectory
    query = """
    SELECT e.*, w.WELLNAME, w.TRAJECTORY
    FROM wells.minerals.ECON_INPUT e
    JOIN wells.minerals.vw_well_input w
    ON e.API_UWI = w.API_UWI
    WHERE e.FAST_EDIT = 1
    ORDER BY w.WELLNAME
    """
    result = session.sql(query).to_pandas()
    return result
    # except Exception as e:
    #     print(f"Error fetching fast edit wells: {e}")
    #     return pd.DataFrame()
get_fast_edit_wells()

c:\Users\es422\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
c:\Users\es422\anaconda3\lib\site-packages\snowflake\connector\options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (19.0.1), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


,API_UWI,LEASE,ECON_SCENARIO,LAST_PROD_DATE,OIL_EMPIRICAL_DECLINE,OIL_PROPHET_DECLINE,OIL_DECLINE_TYPE,OIL_CALC_QI,OIL_CALC_B_FACTOR,OIL_USER_B_FACTOR,...,GAS_USER_DECLINE,GAS_USER_QI,GAS_D_MIN,GAS_Q_MIN,GAS_FCST_YRS,FAST_EDIT,FCST_START_OIL,FCST_START_GAS,WELLNAME,TRAJECTORY
0,42-041-30689,15690,GG,2024-12-01,0.170910,NaN,EXP,68.285714,NaN,0.8,...,0.06,4.0,0.04,NaN,30.0,1,2021-12-01,2021-12-01,"KURTEN, MILTON 1",VERTICAL
1,42-041-31139,25833,GG,2024-11-01,0.200000,NaN,HYP,49.000000,NaN,0.9,...,NaN,NaN,0.04,NaN,30.0,1,2021-11-01,2021-11-01,MAGNOLIA 1,VERTICAL
2,42-041-31139,25833,GG,2024-11-01,0.200000,NaN,HYP,49.000000,NaN,0.9,...,NaN,NaN,0.04,NaN,30.0,1,2021-11-01,2021-11-01,MAGNOLIA 1,HORIZONTAL
3,42-041-31093,17760,GG,2024-12-01,0.372659,NaN,HYP,350.571429,NaN,0.7,...,0.06,1.0,0.04,3.0,30.0,1,2021-12-01,2021-12-01,STRINGFELLOW UNIT 1H,HORIZONTAL


In [3]:
def get_production_data(api_uwi):
    """Get production data for a well"""
    try:
        session = Session.builder.configs(conn).create()

        query = f"""
        SELECT API_UWI, ProducingMonth, LIQUIDSPROD_BBL, GASPROD_MCF
        FROM wells.minerals.raw_prod_data
        WHERE API_UWI = '{api_uwi}'
        ORDER BY ProducingMonth
        """
        
        result = session.sql(query).to_pandas()
        return result
    except Exception as e:
        st.error(f"Error fetching production data: {e}")
        return pd.DataFrame()
    
prd = get_production_data('42-041-30689')
prd['PRODUCINGMONTH'] = pd.to_datetime(prd['PRODUCINGMONTH'])
prd.dtypes


API_UWI                    object
PRODUCINGMONTH     datetime64[ns]
LIQUIDSPROD_BBL           float64
GASPROD_MCF               float64
dtype: object

In [5]:
prd = prd[prd['PRODUCINGMONTH']>='1/1/2020'][['LIQUIDSPROD_BBL','PRODUCINGMONTH']]
formatted_data = "data: [\n" + ",\n".join(
    f"    {{ x: '{date.strftime('%Y-%m-%d')}', y: {value} }}"
    for date, value in zip(prd['PRODUCINGMONTH'], prd['LIQUIDSPROD_BBL'])
) + "\n  ]"

print(formatted_data)

data: [
    { x: '2020-01-01', y: 154.0 },
    { x: '2020-02-01', y: 143.0 },
    { x: '2020-03-01', y: 89.0 },
    { x: '2020-04-01', y: 38.0 },
    { x: '2020-05-01', y: 39.0 },
    { x: '2020-06-01', y: 35.0 },
    { x: '2020-07-01', y: 5.0 },
    { x: '2020-08-01', y: 57.0 },
    { x: '2020-09-01', y: 50.0 },
    { x: '2020-10-01', y: 53.0 },
    { x: '2020-11-01', y: 74.0 },
    { x: '2020-12-01', y: 65.0 },
    { x: '2021-01-01', y: 69.0 },
    { x: '2021-02-01', y: 106.0 },
    { x: '2021-03-01', y: 98.0 },
    { x: '2021-04-01', y: 108.0 },
    { x: '2021-05-01', y: 89.0 },
    { x: '2021-06-01', y: 47.0 },
    { x: '2021-07-01', y: 98.0 },
    { x: '2021-08-01', y: 50.0 },
    { x: '2021-09-01', y: 102.0 },
    { x: '2021-10-01', y: 83.0 },
    { x: '2021-11-01', y: 56.0 },
    { x: '2021-12-01', y: 20.0 },
    { x: '2022-01-01', y: 24.0 },
    { x: '2022-02-01', y: 62.0 },
    { x: '2022-03-01', y: 131.0 },
    { x: '2022-04-01', y: 64.0 },
    { x: '2022-05-01', y: 106.0 },


In [1]:
## this how i created the exploded table


# import snowflake.connector
# import pandas as pd

# # Set up Snowflake connection
# conn = snowflake.connector.connect(
#     user="ELII",
#     password="Elii123456789!",
#     account="CMZNSCB-MU47932",
#     warehouse="COMPUTE_WH",
#     database="WELLS",
#     schema="MINERALS"
# )
# # query = "SELECT * FROM DI_TX_MINERAL_APPRAISALS_2023 LIMIT 10"
# # query = "SELECT SUM(ARRAY_SIZE(SPLIT(API_10, ','))) AS total_well_ids FROM DI_TX_MINERAL_APPRAISALS_2023;"
# # query = "SELECT t.*, TRIM(f.value) AS API_10 FROM MINERALS.DI_TX_MINERAL_APPRAISALS_2023 t,LATERAL FLATTEN(input => SPLIT(t.API_10, ',')) f;"

# # Create a cursor
# cur = conn.cursor()

# # SQL Query to create the exploded table in Snowflake
# create_table_query = """
# CREATE TABLE MINERALS.DI_TX_MINERAL_APPRAISALS_2023_exploded AS
# SELECT 
#     t.*, 
#     TRIM(f.value) AS API_10_exploded
# FROM MINERALS.DI_TX_MINERAL_APPRAISALS_2023 t,
# LATERAL FLATTEN(input => SPLIT(t.API_10, ',')) f;
# """

# # Execute query
# cur.execute(create_table_query)

# # Close connection
# cur.close()
# conn.close()